In [1]:
from datetime import datetime, timedelta
datetime.today().strftime('Data: %Y-%m-%d Hora: %H:%m')

'Data: 2025-03-21 Hora: 10:03'

In [2]:
import pandas as pd
import os
import re

####################################### altera campos negativos entre parenteses para negativo numérico
def transformar_em_negativo(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('(', '-').replace(')', '')
    return valor_str

####################################### Função de formatação de valores 
def transformar_valor(valor):
    if pd.isna(valor):
        return valor
    valor_str = str(valor)
    valor_str = valor_str.replace('.', '').replace(',', '.')
    return float(valor_str)



####################################### Função para remover números e caracteres especiais, mantendo apenas as letras
def remove_special_characters(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)


## FUNÇÃO ABRE ARQUIVO PADRONIZAÇÃO E FAZ JOIN COM A TABELA DE ACORDO COM SEU SHEET

def padronizacao(table,sheet,join_campo):
    caminho_arquivo_padronizacao = "//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Despesas/De_Para/PADRONIZACAO.xls"
    
    df = pd.read_excel(caminho_arquivo_padronizacao, sheet_name=sheet)
    df = df.query("Grupo in ('DESPESA','COMPROMISSADA')")
    df['Descricao'].fillna('', inplace=True)
    df['Descricao'] = df['Descricao'].apply(remove_special_characters) # elimina caracteres exp. e numeros
    df['Descricao'] = df['Descricao'].str.lower()
    df = df[['Descricao','Grupo','Categoria']]
    
    # elimina espaços 
    df_obj = df.select_dtypes('object')
    df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    df = df.drop_duplicates() 
    
    df_merged = pd.merge(table, df, how='inner', on=f'{join_campo}')
    return df_merged

##### POR CodLancamento
def padronizacaoCod(table,sheet,join_campo):
    caminho_arquivo_padronizacao = "//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Despesas/De_Para/PADRONIZACAO.xls"
    
    df = pd.read_excel(caminho_arquivo_padronizacao, sheet_name=sheet)
    df = df.query("Grupo in ('DESPESA','COMPROMISSADA')")
    df['Descricao'].fillna('', inplace=True)
    df['Descricao'] = df['Descricao'].apply(remove_special_characters) # elimina caracteres exp. e numeros
    df['Descricao'] = df['Descricao'].str.lower()
    
    # elimina espaços 
    df_obj = df.select_dtypes('object')
    df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    df = df.drop_duplicates()
    
    df_merged = pd.merge(table, df, how='inner', on=f'{join_campo}')
    return df_merged

# Alimenta o banco de dados
_____

In [3]:
import mysql.connector

def FtDespesas(df):
    # Função para inserir um pedaço do DataFrame na tabela MySQL
    def insert_dataframe_to_mysql(df, connection, table_name):
        cursor = connection.cursor()
        for i, row in df.iterrows():
            sql = f"INSERT INTO {table_name} ({', '.join(row.index)}) VALUES ({', '.join(['%s'] * len(row))})"
            cursor.execute(sql, tuple(row))
        connection.commit()
        cursor.close()

    # Conectar ao banco de dados MySQL
    connection = mysql.connector.connect(
        host='catalise-bi-dados-cluster.cluster-ciaao5zm9beh.sa-east-1.rds.amazonaws.com',
        database='DW_CORPORATIVO',
        user='admin',
        password='yb222KraY7PTN0jbH7P3'
    )

    if connection.is_connected():
        print('Conectado ao MySQL')

        # Dividir o DataFrame em pedaços menores para evitar problemas de memória
        tamanho_do_pedaço = 10000  # Ajuste conforme necessário
        total_registros = len(df)

        for i in range(0, total_registros, tamanho_do_pedaço):
            pedaco_df = df.iloc[i:i+tamanho_do_pedaço]
            insert_dataframe_to_mysql(pedaco_df, connection, 'Ft_Despesas')
            print(f"Registros inseridos: {i + len(pedaco_df)} / {total_registros}")

        print('Concluído')
    else:
        print('Falha na conexão ao MySQL')

    # Fechar a conexão com o banco de dados
    connection.close()

# 

# Singulare

CashStatement.xls

In [4]:
import warnings

# Suprimir apenas DeprecationWarning e FutureWarning
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)



def formata_dataframe(path,nome_do_sheet):
    # Configurar pandas para ignorar SettingWithCopyWarning
    pd.options.mode.chained_assignment = None  # ou 'warn' para mostrar os avisos

    sheet = pd.read_excel(caminho_arquivo_excel, sheet_name=nome_do_sheet)
    ## exclui as 3 primeiras linhas
    df_singulare = sheet.iloc[3:]


    ## cria uma lista com nomes de colunas da primeira linha de valores
    colunas = []
    for i in range(len(df_singulare.columns)):
        x = df_singulare.iloc[0][i]
        colunas.append(x) 

    ## Renomeia colunas usando a linha 
    df_singulare.columns = colunas

    ## será usado este comapo apenas para o join 
    df_singulare['Descricao'] = df_singulare['Histórico'].apply(remove_special_characters) # elimina caracteres exp. e numeros

    # ## Obtem nome do fundo 
    NmFundo = sheet[:3]['Demonstrativo de Caixa'][2].replace('Cliente: ', '')
    df_singulare['NmFundo'] = NmFundo

    ## exclui a primeira linha 
    df_singulare = df_singulare.iloc[1:]

    # Preencher os valores NaN com o valor anterior
    df_singulare['Data'] = df_singulare['Data'].fillna(method='ffill')

    ## renomeia colunas para banco de dados
    df_singulare = df_singulare[['Data','NmFundo','Histórico','Entradas','Saídas','Descricao']]
    df_singulare['Descricao'] = df_singulare['Descricao'].str.lower()
    
    

    #################################################### FAZ A NORMALIZAÇÃO COM O DE-PARA#####################################
    # aplica a função
    df_singulare_v2 = padronizacao(df_singulare,"SINGULARE","Descricao")


    ## converte data em datetime
    date_columns = ['Data']
    df_singulare_v2[date_columns] = df_singulare_v2[date_columns].apply(pd.to_datetime, format="%d/%m/%Y")

    # ## Valores numéricos
    # ## valores negativos do arquivo representados entre parenteses

    df_singulare_v2['Entradas'].fillna(0, inplace=True)
    df_singulare_v2['Saídas'].fillna(0, inplace=True)
    
    columns_to_convert = ['Entradas', 'Saídas']

    ## 
    for column in columns_to_convert:
        df_singulare_v2[column] = df_singulare_v2[column].apply(transformar_em_negativo)

    ## converte campos em float
    for column in columns_to_convert:
        df_singulare_v2[column] = df_singulare_v2[column].apply(transformar_valor)
        
    # ## troca NaN por zero
    df_singulare_v2['VlrLancamento'] = df_singulare_v2['Entradas']+df_singulare_v2['Saídas']
    

    ## exclui coluna
    df_singulare_v2.drop(columns=['Entradas','Saídas','Descricao'], inplace=True)
    df_singulare_v2.columns = ['DtReferencia','NmFundo','Descricao','Grupo','Categoria','VlrLancamento']

    return df_singulare_v2






######################################## aplica a função
import os
import pandas as pd

# Caminho do diretório onde estão os arquivos
diretorio = '//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Despesas/SINGULARE/'

# Loop para percorrer todos os arquivos no diretório
for arquivo in os.listdir(diretorio):
    # Verifica se o arquivo contém "CashStatement" no nome e termina com .xls ou .xlsx
    if 'CashStatement' in arquivo and arquivo.endswith(('.xls', '.xlsx')):
        caminho_arquivo_excel = os.path.join(diretorio, arquivo)
        
        # Ler o arquivo Excel
#         excel_file = pd.ExcelFile(caminho_arquivo_excel)
        # Ler o arquivo Excel usando o bloco "with"
        with pd.ExcelFile(caminho_arquivo_excel) as excel_file:

            # Obter os nomes das sheets
            sheets = excel_file.sheet_names

            # Lista para armazenar os dataframes
            dfs = []

            # Iterar sobre as sheets e aplicar a função de formatação
            for i in range(len(sheets)):
                df = formata_dataframe(caminho_arquivo_excel, sheets[i])
                dfs.append(df)

        # Concatenar todos os dataframes em um único dataframe
        df_singulare = pd.concat(dfs, ignore_index=True)

        # Tratar dados
        df_singulare.fillna('', inplace=True)
        df_singulare['NmFundo'] = df_singulare['NmFundo'].str.strip()

        # Tratativas para nomes específicos
        df_singulare.loc[df_singulare['NmFundo'] == 'AGROFORTE FIDC - SUBORDINADA', 'NmFundo'] = 'FIDC AGROFORTE'
        df_singulare.loc[df_singulare['NmFundo'] == 'APG FIDC - SUBORDINADA', 'NmFundo'] = 'FIDC APG'
        df_singulare.loc[df_singulare['NmFundo'] == 'BRISTOL FIDC MULTISSETORIAL', 'NmFundo'] = 'FIDC BRISTOL'

        # (Aqui você pode adicionar qualquer lógica de processamento ou salvamento para cada arquivo encontrado)
        print(f"Processado: {arquivo}")


WARNING *** file size (92858) not 512 + multiple of sector size (512)
WARNING *** file size (92858) not 512 + multiple of sector size (512)
WARNING *** file size (92858) not 512 + multiple of sector size (512)
WARNING *** file size (92858) not 512 + multiple of sector size (512)
WARNING *** file size (92858) not 512 + multiple of sector size (512)
WARNING *** file size (92858) not 512 + multiple of sector size (512)
WARNING *** file size (92858) not 512 + multiple of sector size (512)
WARNING *** file size (92858) not 512 + multiple of sector size (512)
WARNING *** file size (92858) not 512 + multiple of sector size (512)
WARNING *** file size (92858) not 512 + multiple of sector size (512)
WARNING *** file size (92858) not 512 + multiple of sector size (512)
Processado: CashStatement.xls


In [5]:
df_singulare.NmFundo.drop_duplicates(inplace=True)

In [6]:
df_singulare

,DtReferencia,NmFundo,Descricao,Grupo,Categoria,VlrLancamento
0,2025-03-05,FIDC APG,TARIFA DE TED -REF.03/2025,DESPESA,TARIFA BANCARIA,-31.75
1,2025-03-06,FIDC APG,Consultoria,DESPESA,TAXA CONSULTORIA,-150000.00
2,2025-03-06,FIDC APG,REEMBOLSO,DESPESA,AJUSTE,15420.71
3,2025-03-06,FIDC APG,TARIFA DE TED -REF.03/2025,DESPESA,TARIFA BANCARIA,-38.10
4,2025-03-07,FIDC APG,REEMBOLSO,DESPESA,AJUSTE,1090.93
5,2025-03-07,FIDC APG,TARIFA DE TED -REF.03/2025,DESPESA,TARIFA BANCARIA,-44.45
6,2025-03-10,FIDC APG,TARIFA DE TED -REF.03/2025,DESPESA,TARIFA BANCARIA,-6.35
7,2025-03-11,FIDC APG,TARIFA DE TED -REF.03/2025,DESPESA,TARIFA BANCARIA,-19.05
8,2025-03-11,FIDC APG,Taxa de Administração Bruta,DESPESA,TAXA ADMINISTRACAO,-14111.29
9,2025-03-11,FIDC APG,Taxa de Custódia Bruta,DESPESA,TAXA CUSTODIA,-1567.92


In [7]:
df_singulare.count()

DtReferencia     32
NmFundo          32
Descricao        32
Grupo            32
Categoria        32
VlrLancamento    32
dtype: int64

In [8]:
print(f"Min: {df_singulare.DtReferencia.min()}\nMax: {df_singulare.DtReferencia.max()}")

Min: 2025-03-05 00:00:00
Max: 2025-03-19 00:00:00


### Insere no banco

In [9]:
FtDespesas(df_singulare)

Conectado ao MySQL
Registros inseridos: 32 / 32
Concluído


In [10]:
# Limpa pasta 
import os
 
dir = diretorio
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

 ________

# Daycoval

download.csv

In [11]:
######################### Desaativa avisos ####################################
import warnings

# Suprimir apenas DeprecationWarning e FutureWarning
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
#################################################################

def formata_arq_daycoval(caminho_pasta,contador):

    # Caminho da pasta com os arquivos
    arquivos = os.listdir(caminho_pasta)

    # Encontra arquivo 
    file = list(filter(lambda x: 'Demonstrativo de Caixa' in x, arquivos))
    df_daycoval = pd.DataFrame()


    # Lê o arquivo CSV
    df_daycoval = pd.read_csv(os.path.join(caminho_pasta, file[contador]), delimiter=';', encoding='latin-1', dtype='str')
    df_daycoval=df_daycoval.iloc[::-1].reset_index(drop=True)

    ## cria uma lista com nomes de colunas da primeira linha de valores
    colunas = []
    for i in range(len(df_daycoval.columns)):
        x = df_daycoval.iloc[0][i]
        colunas.append(x) 

    ## Renomeia colunas usando a linha 
    df_daycoval.columns = colunas

    ## seleciona colunas importantes
    df_daycoval = df_daycoval[['DtExibicao','NmCrt','Hist','TitCP','VlDeb','VlCred']]

    ## renomeia
    df_daycoval.columns = ['DtReferencia','NmFundo','Descricao','CodLancamento','VlrEntrada','VlrSaida']
    
#     df_daycoval['Descricao'] = df_daycoval['Descricao'].str.lower()


    # elimina espaços 
    df_obj = df_daycoval.select_dtypes('object')
    df_daycoval[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

    ## aplica join com o DE-PARA
    df_daycoval = padronizacaoCod(df_daycoval,"DAYCOVAL","CodLancamento")

    df_daycoval = df_daycoval[['DtReferencia','NmFundo','Descricao_x','CodLancamento','VlrEntrada','VlrSaida','Grupo','Categoria']]
    df_daycoval.rename(columns={'Descricao_x':"Descricao"}, inplace=True)
    
#     ## elimina caracteres exp. e numeros
#     df_daycoval['Descricao'] = df_daycoval['Histórico'].apply(remove_special_characters) 

    
    ## tratativas de colunas
    ## Converte a coluna 'DtReferencia' para datetime
    df_daycoval['DtReferencia'] = pd.to_datetime(df_daycoval['DtReferencia'], format="%d/%m/%Y")# "%d/%m/%Y"

    df_daycoval['VlrEntrada'].fillna(0, inplace=True)
    df_daycoval['VlrSaida'].fillna(0, inplace=True)
    
    ## converter valores
    columns_to_convert = ['VlrEntrada','VlrSaida']

    ## converte campos em float
    for column in columns_to_convert:
        df_daycoval[column] = df_daycoval[column].apply(transformar_valor)


    # Aplica a transformação para a coluna 'Saída'
    df_daycoval.loc[df_daycoval['VlrSaida'] > 0, 'VlrSaida'] *= -1
    
    ## Calulo
    df_daycoval['VlrLancamento'] = (df_daycoval['VlrEntrada'] + df_daycoval['VlrSaida'])
    
    ## campos vazios podem ser interpretados por funções como numérico
    df_daycoval.fillna('', inplace=True)
    
    return df_daycoval

In [12]:
# pip install openpyxl xlrd
import os
import pandas as pd

# Configurar pandas para ignorar SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # ou 'warn' para mostrar os avisos

# Caminho da pasta onde estão os arquivos
caminho_pasta = "//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Despesas/DAYCOVAL"

# Lista os arquivos no diretório
arquivos = os.listdir(caminho_pasta)

# Filtra os arquivos que contêm 'Demonstrativo de Caixa' no nome
arquivos_filtrados = [arq for arq in arquivos if 'Demonstrativo de Caixa' in arq]

# Inicializar uma lista vazia para armazenar os DataFrames
df_list = []

# Loop através da lista de arquivos filtrados
for i, arquivo in enumerate(arquivos_filtrados):  # Usar enumerate para obter o índice
    caminho_arquivo = os.path.join(caminho_pasta, arquivo)
    
    # Passa o caminho da pasta e o índice para a função
    df_list.append(formata_arq_daycoval(caminho_pasta, i))  # Passa o caminho da pasta e o índice

# Concatenar todos os DataFrames em um único DataFrame
df_daycoval = pd.concat(df_list, ignore_index=True)

# Excluir colunas indesejadas
df_daycoval.drop(columns=['VlrEntrada', 'VlrSaida', 'CodLancamento'], inplace=True)

# Normaliza Nome FIDC
df_daycoval['NmFundo'] = df_daycoval['NmFundo'].str.strip()  # remove espaços

# Dicionário para mapeamento dos nomes
mapeamento_nomes = {
    'BLUE ROCKET  SUB': 'FIDC B ROCKET',
    'FIDC GLOBAL FUTU': 'FIDC GLOBAL FUTURO',
    'IPE FIDC NP': 'FIDC IPE',
    'PRIME AGRO FIDC': 'FIDC PRIME AGRO',
    'ÁGIS 2 FIDC': 'FIDC AGIS 2',
    'ÁGIS - FIDC NP': 'FIDC AGIS',
    'ALBAREDO FIDC': 'FIDC ALBAREDO',
    'SC FUNDO DE INVESTIMENTO EM DC - LR': 'FIDC SC',
    'PRIME AGRO FIDC RL': 'FIDC PRIME AGRO',
    'Z INVEST FIDC': 'FIDC Z INVEST',
    'GLOBAL FUTURA FIDC RL': 'FIDC GLOBAL FUTURA',
    'BASÃ- FUNDO DE INVESTIMENTO EM DIREITOS': 'FIDC BASÃ',
    'IPE-FUNDO DE INVESTIMENTO EM DIREITOS NP': 'FIDC IPE',
    'SDL - FUNDO DE INVESTIMENTO EM DIREITOS': 'FIDC SDL',
    'K-FINANCE FIDC NP - SUBORDINADA': 'FIDC K-FINANCE',
    'AGROCETE FUNDO DE INVESTIMENTO EM DIREIT': 'FIDC AGROCETE',
    'AF6 FIDC - NP': 'FIDC AF6',
    'BELL FUNDO DE INVESTIMENTO EM DIREITOS C': 'FIDC BELL',
    'VERGINIA FUNDO DE INVESTIMENTO EM DIREIT': 'FIDC VERGINIA',
    'CREDIAL BANK PAN FIDC': 'FIDC CREDIAL',
    'KÉRDOS FUNDO DE INVESTIMENTO EM DIREITOS': 'FIDC KÉRDOS',
    'CAPITALIZA FUNDO DE INVESTIMENTO EM DIRE': 'FIDC CAPITALIZA',
    'FUTURO CAPITAL FUNDO DE INVESTIMENTO EM': 'FIDC FUTURO CAPITAL',
    'BONTEMPO FIDC  - RL': 'FIDC BONTEMPO',
    'VELSO - FUNDO DE INVESTIMENTO EM DIREITO': 'FIDC VELSO',
    'GREENWOOD FIDC NP': 'FIDC GREENWOOD',
    'NINE CAPITAL FIDC': 'FIDC NINE CAPITAL',
    'BEFIC FUNDO DE INVESTIMENTO EM DIREITOS': 'FIDC BEFIC',
    'CREDILOG - FUNDO DE INVESTIMENTO EM DIRE': 'FIDC CREDILOG',
    'VISHNU FUNDO DE INVESTIMENTO EM DIREITOS': 'FIDC VISHNU',
    'MASTRENN FIDC - RL': 'FIDC MASTRENN',
    'BLUE ROCKET FIDC  - SUBORDINADA': 'FIDC BLUE ROCKET S',
    'USECORP CATÁLISE FIDC': 'FIDC USECORP CATÁLISE',
    'FIDC F2 BANK SUBORDINADA': 'FIDC F2 BANK',
    'CREDILOG II - FUNDO DE INVESTIMENTO EM D': 'FIDC CREDILOG II',
    'PRIME AGRO FIDC RL - SEN                ': 'FIDC PRIME AGRO SR',
    '3RD - FUNDO DE INVESTIMENTO EM DIREITOS': 'FIDC 3RD',
    'DBANK FUNDO DE INVESTIMENTO EM DIREITOS': 'FIDC DBANK',
    'ARABAN FIC FIM RESPONSABILIDADE LIMITAD': 'FICFIM ARABAN',
    'MACANAN FIC FIM RESPONSABILIDADE LIMITAD': 'FICFIM MACANAN',
    'CONDOBEM FUNDO DE INVESTIMENTO EM DIREIT': 'FIDC CONDOBEM',
    'TORONTO  FIDC RESP LIMITADA': 'FIDC TORONTO',
    'MALBEC FIDC - RL': 'FIDC MALBEC',
    'VITTRA FUNDO DE INVESTIMENTO EM DIREITOS': 'FIDC VITTRA',
    'EJM FUNDO DE INVESTIMENTO EM DC - RL': 'FIDC EJM',
    'SMT AGRO FUNDO DE INVESTIMENTO EM DIREIT': 'FIDC SMT AGRO',
    'CATALISE FIC FIDC - RL':  'FICFIDC CATALISE',
    'NR11 FUNDO DE INVESTIMENTO EM DIREITOS C': 'FIDC NR11',
    'ANVERES FUNDO DE INVESTIMENTO EM DIREITO': 'FIDC ANVERES',
    'ANIL FUNDO DE INVESTIMENTO EM DIREITOS C': 'FIDC ANIL'
    
}

# Aplicar o mapeamento para normalizar os nomes
df_daycoval['NmFundo'].replace(mapeamento_nomes, inplace=True)

# Remover duplicados novamente após a normalização
# df_daycoval.drop_duplicates(subset=['NmFundo', 'DtReferencia'], inplace=True)

# Mostrar os dados únicos após a normalização
df_daycoval[['NmFundo']].drop_duplicates()


,NmFundo
0,FIDC MALBEC
2,FIDC F2 BANK
10,FIDC LIBRA CONSIGNADO
12,FIDC CREDIAL
16,FIDC KÉRDOS
19,FIDC VERGINIA
23,FIDC BELL
27,FIDC AF6
31,FIDC BLUE ROCKET S
35,FIDC SDL


In [13]:
df_daycoval

,DtReferencia,NmFundo,Descricao,Grupo,Categoria,VlrLancamento
0,2025-03-19,FIDC MALBEC,DOC/TED INTERNET TED INTERNET,DESPESA,TARIFA BANCARIA,-6.55
1,2025-03-19,FIDC MALBEC,DOC/TED INTERNET TED INTERNET,DESPESA,TARIFA BANCARIA,-6.55
2,2025-03-19,FIDC F2 BANK,DOC/TED INTERNET TED INTERNET,DESPESA,TARIFA BANCARIA,-6.55
3,2025-03-19,FIDC F2 BANK,DOC/TED INTERNET TED INTERNET,DESPESA,TARIFA BANCARIA,-6.55
4,2025-03-19,FIDC F2 BANK,TARIFA AUTORIZ COBRANCA TIT.BX.D,DESPESA,TARIFA BANCARIA,-7.02
...,...,...,...,...,...,...
73,2025-03-19,FIDC AGIS,Compra Cota CRT 969583 DAY T. PUB. II,COMPROMISSADA,COMPROMISSADA,-22390028.11
74,2025-03-19,FIDC ALBAREDO,Venda Cota CRT 969583 DAY T. PUB. II,COMPROMISSADA,COMPROMISSADA,90571.22
75,2025-03-19,FIDC ALBAREDO,Venda Cota CRT 969583 DAY T. PUB. II,COMPROMISSADA,COMPROMISSADA,90571.22
76,2025-03-19,FIDC ALBAREDO,Compra Cota CRT 969583 DAY T. PUB. II,COMPROMISSADA,COMPROMISSADA,-78511.60


In [14]:
print(f"Min: {df_daycoval.DtReferencia.min()}\nMax: {df_daycoval.DtReferencia.max()}")

Min: 2025-03-19 00:00:00
Max: 2025-03-19 00:00:00


In [15]:
df_daycoval
 

,DtReferencia,NmFundo,Descricao,Grupo,Categoria,VlrLancamento
0,2025-03-19,FIDC MALBEC,DOC/TED INTERNET TED INTERNET,DESPESA,TARIFA BANCARIA,-6.55
1,2025-03-19,FIDC MALBEC,DOC/TED INTERNET TED INTERNET,DESPESA,TARIFA BANCARIA,-6.55
2,2025-03-19,FIDC F2 BANK,DOC/TED INTERNET TED INTERNET,DESPESA,TARIFA BANCARIA,-6.55
3,2025-03-19,FIDC F2 BANK,DOC/TED INTERNET TED INTERNET,DESPESA,TARIFA BANCARIA,-6.55
4,2025-03-19,FIDC F2 BANK,TARIFA AUTORIZ COBRANCA TIT.BX.D,DESPESA,TARIFA BANCARIA,-7.02
...,...,...,...,...,...,...
73,2025-03-19,FIDC AGIS,Compra Cota CRT 969583 DAY T. PUB. II,COMPROMISSADA,COMPROMISSADA,-22390028.11
74,2025-03-19,FIDC ALBAREDO,Venda Cota CRT 969583 DAY T. PUB. II,COMPROMISSADA,COMPROMISSADA,90571.22
75,2025-03-19,FIDC ALBAREDO,Venda Cota CRT 969583 DAY T. PUB. II,COMPROMISSADA,COMPROMISSADA,90571.22
76,2025-03-19,FIDC ALBAREDO,Compra Cota CRT 969583 DAY T. PUB. II,COMPROMISSADA,COMPROMISSADA,-78511.60


# 

### Insere no Banco

In [16]:
#####################
FtDespesas(df_daycoval)

Conectado ao MySQL
Registros inseridos: 78 / 78
Concluído


In [17]:
# Limpa pasta 
import os
 
dir = caminho_pasta
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

___________

# 

# Master

PTR_49002_20240705_141213.csv

In [18]:
def formata_arq_master(caminho_pasta,contador):

    arquivos = os.listdir(caminho_pasta)

    # Encontra arquivo AGIS
    file = list(filter(lambda x: 'PTR' in x, arquivos))
    df_master = pd.DataFrame()


    # Lê o arquivo CSV
    df_master = pd.read_csv(os.path.join(caminho_pasta, file[contador]), delimiter=';', encoding='latin-1', dtype='str')


    # Converte a coluna 'DtReferencia' para datetime
    df_master['DATALANCAMENTO'] = pd.to_datetime(df_master['DATALIQUIDACAO'], format="%d/%m/%Y")

    # converter valores
    columns_to_convert = ['CREDITO', 'DEBITO']

    ## 
    for column in columns_to_convert:
        df_master[column] = df_master[column].apply(transformar_em_negativo)

    ## converte campos em float
    for column in columns_to_convert:
        df_master[column] = df_master[column].apply(transformar_valor)

    df_master['CREDITO'].fillna(0, inplace=True)
    df_master['DEBITO'].fillna(0, inplace=True)
    
    # ## troca NaN por zero
    df_master['VlrLancamento'] = df_master['CREDITO']+df_master['DEBITO']
    

    df_master.rename(columns={"CODIGOLANCAMENTO":'CodLancamento'}, inplace=True)

    ## elimina caracteres exp. e numeros
    df_master['Descricao'] = df_master['HISTORICO'].apply(remove_special_characters) 

    df_master['Descricao'].fillna('', inplace=True)
    df_master['Descricao'] = df_master['Descricao'].str.lower()

    # aplica a função join
    df_master = padronizacao(df_master,"MASTER","Descricao")

    # ## renomeia
    df_master = df_master[['DATALANCAMENTO','CARTEIRA','Descricao','VlrLancamento', 'Grupo','Categoria','HISTORICO']]
    df_master.columns = ['DtReferencia','NmFundo','Descricao','VlrLancamento','Grupo','Categoria','HISTORICO']
#     df_master.drop_duplicates(inplace=True)
    
    return df_master

In [19]:
# Configurar pandas para ignorar SettingWithCopyWarning
pd.options.mode.chained_assignment = None  # ou 'warn' para mostrar os avisos

caminho_pasta = "//172.31.8.209/Files/CataliseInvestimentos/14. BigData Catalise/Staging/Despesas/MASTER"

# Caminho da pasta com os arquivos
arquivos = os.listdir(caminho_pasta)

# Encontra arquivo 
file = list(filter(lambda x: 'PTR' in x, arquivos))



################################################################
# Inicializar uma lista vazia para armazenar os DataFrames
df_list = []

# Loop através da lista de arquivos filtrados
for i in range(len(file)):
    # Chamar a função formata_arq_master e adicionar o DataFrame resultante à lista
    df_list.append(formata_arq_master(caminho_pasta, i))

# Concatenar todos os DataFrames em um único DataFrame
df_master = pd.concat(df_list, ignore_index=True)

df_master.drop(columns={'Descricao'}, inplace=True)

df_master.rename(columns={"HISTORICO":'Descricao'}, inplace=True)
# df_master.drop_duplicates(inplace=True)


print(df_master[['NmFundo','DtReferencia']].drop_duplicates())

print(f"\n{len(df_master)} Linhas")

           NmFundo DtReferencia
0     FIDC TOPCRED   2025-03-18
3     FIDC TRACTOR   2025-03-18
4         FIDC DFC   2025-03-18
5   FIDC FERPAR NP   2025-03-18
6     FIDC BINVEST   2025-03-18
7   FIDC BETTA SUB   2025-03-18
9     FIDC TOPCRED   2025-03-19
11        FIDC DFC   2025-03-19
13   FIDC UNITY NP   2025-03-19
14     FIDC PEROLA   2025-03-19
15  FIDC FERPAR NP   2025-03-19
16    FIDC BINVEST   2025-03-19
17  FIDC BETTA SUB   2025-03-19

18 Linhas


In [20]:
# Verificar linhas duplicadas considerando colunas específicas
duplicadas = df_master.duplicated(subset=['DtReferencia', 'NmFundo', 'Descricao', 'VlrLancamento', 'Grupo', 'Categoria'])

# Exibir linhas duplicadas
linhas_duplicadas = df_master[duplicadas]
linhas_duplicadas

,DtReferencia,NmFundo,VlrLancamento,Grupo,Categoria,Descricao


### Insere no Banco

In [21]:
print(f"Min: {df_master.DtReferencia.min()}\nMax: {df_master.DtReferencia.max()}")

Min: 2025-03-18 00:00:00
Max: 2025-03-19 00:00:00


In [22]:
df_master.query("NmFundo == 'FIDC TOPCRED' and DtReferencia == '2024-08-09'")

,DtReferencia,NmFundo,VlrLancamento,Grupo,Categoria,Descricao


In [23]:
FtDespesas(df_master)

Conectado ao MySQL
Registros inseridos: 18 / 18
Concluído


In [24]:
# Limpa pasta 
import os
 
dir = caminho_pasta
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))